In [1]:
import requests
import os
from IPython.display import display
from ipywidgets import widgets
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

generate_data_button = widgets.Button(description="Generate data")

df = []

def get_host():
    host = os.getenv("KMEANS_BACKEND_HOST")
    if not host:
        host = "localhost"
    return host

def get_port():
    port = os.getenv("KMEANS_BACKEND_PORT")
    if not port:
        port = 11111
    return port


def generate_data(b):
    global df
    host = get_host()
    port = get_port()
    url = 'http://{}:{}/generatedata/default'.format(host, port)
    print(url)
    headers = {'Content-Type':'application/json'}
    body = "{\"clusterNum\": 3," \
           " \"bounds\": [[100, 250],[100, 250],[350, 750],[350, 750],[1000, 2500],[1000, 2500]]}"
    data = {'headers': headers, 'body': body}
    response = requests.get(url, data)
    input_data = response.json()['data']
    df = pd.DataFrame(input_data, columns =['x', 'y'], dtype = int)
    show_data()

def show_data():
    fig = plt.figure(figsize=(20, 15))
    ax = fig.add_subplot(111)
    ax.scatter(x=df["x"],
               y=df["y"],
               s=30,
               c='b',
               label='Generated data')

display(generate_data_button)

generate_data_button.on_click(generate_data)


Button(description='Generate data', style=ButtonStyle())